In [ ]:
pip install tensorflow transformers

In [ ]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.7 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import pipeline
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset
from datasets import load_dataset
from sklearn.model_selection import train_test_split


In [ ]:

sst_dataset = load_dataset("sst")
print(sst_dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for sst contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/sst
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/8544 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1101 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2210 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'tokens', 'tree'],
        num_rows: 8544
    })
    validation: Dataset({
        features: ['sentence', 'label', 'tokens', 'tree'],
        num_rows: 1101
    })
    test: Dataset({
        features: ['sentence', 'label', 'tokens', 'tree'],
        num_rows: 2210
    })
})


In [ ]:
import numpy as np
import numpy as np
train_texts, train_labels, test_texts, test_labels = sst_dataset['train']['sentence'], sst_dataset['train']['label'],sst_dataset['test']['sentence'], sst_dataset['test']['label']

bin_edges = [0.25, 0.50, 0.75]

label_categories = np.digitize(train_labels, bin_edges)
#  digitize function into bins
train_label_categories = np.digitize(train_labels, bin_edges)
test_label_categories = np.digitize(test_labels, bin_edges)

for i in range(1,20):
  print(f"text {i}: "+ train_texts[i] +" L1: "+str(train_labels[i])+ " L2: "+str(label_categories[i]),end="\n")

text 1: The gorgeously elaborate continuation of `` The Lord of the Rings '' trilogy is so huge that a column of words can not adequately describe co-writer\/director Peter Jackson 's expanded vision of J.R.R. Tolkien 's Middle-earth . L1: 0.833329975605011 L2: 3
text 2: Singer\/composer Bryan Adams contributes a slew of songs -- a few potential hits , a few more simply intrusive to the story -- but the whole package certainly captures the intended , er , spirit of the piece . L1: 0.625 L2: 2
text 3: You 'd think by now America would have had enough of plucky British eccentrics with hearts of gold . L1: 0.5 L2: 2
text 4: Yet the act is still charming here . L1: 0.7222200036048889 L2: 2
text 5: Whether or not you 're enlightened by any of Derrida 's lectures on `` the other '' and `` the self , '' Derrida is an undeniably fascinating and playful fellow . L1: 0.833329975605011 L2: 3
text 6: Just the labour involved in creating the layered richness of the imagery in this chiaroscuro of ma

In [ ]:
# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
model = BertForSequenceClassification.from_pretrained('bert-large-uncased', num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:

train_tokenized = tokenizer(train_texts, padding=True, truncation=True, return_tensors='pt')
test_tokenized = tokenizer(test_texts, padding=True, truncation=True, return_tensors='pt')


train_l = torch.tensor(train_label_categories)
test_l = torch.tensor(test_label_categories)

train_d = TensorDataset(train_tokenized['input_ids'], train_tokenized['attention_mask'], train_l)
test_d = TensorDataset(test_tokenized['input_ids'], test_tokenized['attention_mask'], test_l)

train_dataloader = DataLoader(train_d, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_d, batch_size=16, shuffle=False)


#test_labels = torch.tensor(test_labels)

In [ ]:
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm

train_accuracies = []
# Set up training parameters
optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 2
scheduler = StepLR(optimizer, step_size=1, gamma=0.9)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Fine-tuning loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    pos = epoch + 1
    progress_bar = tqdm(enumerate(train_dataloader), total=len(train_dataloader), desc=f"Epoch {epoch + 1}/{num_epochs}")

    for batch_idx, batch in progress_bar:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

        progress_bar.set_postfix({'Training Loss': total_loss / (batch_idx + 1)})

    average_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {average_loss}")
    train_accuracies.append(average_loss)

# Save the fine-tuned model
model.save_pretrained("./fine_tuned_bert_sst")
tokenizer.save_pretrained("./fine_tuned_bert_sst")


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1/2: 100%|██████████| 534/534 [05:59<00:00,  1.49it/s, Training Loss=0.396]


Epoch 1/2, Training Loss: 0.39626566082071724


Epoch 2/2: 100%|██████████| 534/534 [05:59<00:00,  1.49it/s, Training Loss=0.262]


Epoch 2/2, Training Loss: 0.26197408449100085


('./fine_tuned_bert_sst/tokenizer_config.json',
 './fine_tuned_bert_sst/special_tokens_map.json',
 './fine_tuned_bert_sst/vocab.txt',
 './fine_tuned_bert_sst/added_tokens.json')

In [ ]:
model = BertForSequenceClassification.from_pretrained("./fine_tuned_bert_sst")
tokenizer = BertTokenizer.from_pretrained("./fine_tuned_bert_sst")
model.to(device)  # Make sure the model is on the same device as the data

# Set  model to eval
model.eval()
correct = 0
samples = 0

# Evaluate  test
with torch.no_grad():
    for batch in test_dataloader:s
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1)

        correct += (predictions == labels).sum().item()
        samples += len(predictions)

# accuracy
accuracy = correct / samples
print(f"Test Accuracy: {accuracy}")

Test Accuracy: 0.6036199095022624
